In [1]:
import boto3
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
import regex as re
from spellchecker import SpellChecker
import nltk

# Amazon Textract (OCR)

In [2]:
# Configure access to textract
cd = pd.read_csv("Administrator_accessKeys.csv")
client = boto3.client(
    'textract',
    aws_access_key_id=cd["Access key ID"][0],
    aws_secret_access_key=cd["Secret access key"][0],
    region_name = "us-east-2"
)

In [6]:
# Get image files from s3
bucket = "oligarchy2021"

clt = boto3.client(
    's3',
    aws_access_key_id=cd["Access key ID"][0],
    aws_secret_access_key=cd["Secret access key"][0],
    region_name = "us-east-2"
)
clt.get_paginator('list_objects_v2')
paginator = clt.get_paginator('list_objects_v2')
result = paginator.paginate(Bucket= bucket)
for page in result:
    if "Contents" in page:
        image_files = [key[ "Key" ] for key in page[ "Contents"]]

KeyboardInterrupt: 

In [18]:
def extract_text(bucket, name):
    """Extract text from image file.

    Input: 
    bucket(str): Name of bucket in S3
    name(str): Image file name
    
    Returns: 
    (str): Document text
    """
    response = client.detect_document_text(
        Document={'S3Object': {'Bucket': bucket, 'Name': name}})

    text = []

    for item in response["Blocks"]:
        if item["BlockType"] == "LINE":
            text.append(item["Text"])

    # Remove lines with page headers
    if text[0].isupper():
        if len(text[0])<8 or fuzz.partial_ratio(text[0], book_title) >50:
            print(text.pop(0))
        
    return " ".join(text) + " "

In [5]:
book_title = "QUIEN ES QUIEN EN COLOMBIA"
original_text = {}

#for fle in image_files:
    page = re.sub("[^0-9]", "", fle)
    original_text[page] = original_text.get(page, "") + " "+ extract_text(bucket, fle) 
    #original_text.setdefault(page, "").append(" "+ extract_text(bucket,fle))

EN EN COLOMBIA
QUIEN ES QUTEN
EN COLOMBIA
EN EN COLOMBIA
EN EN COLOMBIA
UTEN EN COLOMBIA
EN EN COLOMBIA
QUIEN ES QUI
EN EN COLOMBIA
N EN COLOMBIA
EN EN COLOMBIA
EN EN COLOMBIA
EN EN COLOMBIA
EN EN COLOMBIA
IEN EN COLOMBIA
:N EN COLOMBIA
EN EN COLOMBIA
EN EN COLOMBIA
N EN COLOMBIA
N EN COLOMBIA
EN EN COLOMBIA
N EN COLOMBIA
N EN COLOMBIA
I EN COLOMBIA
N EN COLOMBIA
EN COLOMBIA
N EN COLOMBIA
1 EN COLOMBIA
N EN COLOMBIA
N EN COLOMBIA
EN EN COLOMBIA
EN COLOMBIA
EN EN COLOMBIA
N EN COLOMBIA
N EN COLOMBIA
A R T I
S T A S
N EN COLOMBIA
:N EN COLOMBIA
N EN COLOMBIA
IN EN COLOMBIA
A
O R E S
N EN COLOMBIA
QUIEN ES QUIEN
EN COLOMBIA
N EN COLOMBIA
EN COLOMBIA
N EN COLOMBIA
N COLOMBIA
ISTICOS
N EN COLOMBIA
EN COLOMBIA
EN EN COLOMBIA
N EN COLOMBIA
N EN COLOMBIA
IN EN COLOMBIA
N EN COLOMBIA
EN COLOMBIA
QUIEN ES QUIE!
N EN COLOMBIA
EN COLOMBIA
N EN COLOMBIA
N EN COLOMBIA
N EN COLOMBIA
EN COLOMBIA
N EN COLOMBIA
ECONO.
MISTAS
EN EN COLOMBIA
EN EN COLOMBIA
ECONOMISTAS
N EN COLOMBIA
N EN COLOMBIA
ECONOMIST

In [6]:
# Save original text as json file
import json
with open('data.json', 'w') as f:
    json.dump(original_text, f)

# Reading Missed Pages

In [13]:
# Get image files from s3
bucket = "oligarchy2021"

clt = boto3.client(
    's3',
    aws_access_key_id=cd["Access key ID"][0],
    aws_secret_access_key=cd["Secret access key"][0],
    region_name = "us-east-2"
)
clt.get_paginator('list_objects_v2')
paginator = clt.get_paginator('list_objects_v2')
result = paginator.paginate(Bucket= bucket, Prefix = "missingpages")
for page in result:
    if "Contents" in page:
        image_files = [key[ "Key" ] for key in page[ "Contents"]]

In [19]:
book_title = "QUIEN ES QUIEN EN COLOMBIA"
missing_text = {}

#for fle in image_files[1:]:
    page = re.sub("[^0-9]", "", fle)
    missing_text[page] = missing_text.get(page, "") + " "+ extract_text(bucket, fle) 

EN EN COLOMBIA
ASTICOS


# Split Text into Sections

In [33]:
import json
with open('data.json',) as f:
    original_text = json.load(f)

In [34]:
# merged_text = original_text | missing_text
# original_text = dict(sorted(merged_text.items()))

In [36]:
book_title = "QUIEN ES QUIEN EN COLOMBIA"

# Permutations of header in book
perm = [book_title[:i] for i in np.arange(15, 5,-1)]+ [book_title[6:i] for i in np.arange(15, 8,-1)]+ [book_title[i:] for i in np.arange(5,len(book_title)-9)]
full_text = ""
full_name = []
page_no = []

# Extract full names of entries in book
for pg in original_text:
    t = original_text[pg]
    t = re.sub(r'(?<=[.])(?=[^\s])', ". ", t) #add space after fullstops
    t = re.sub(" -", " ", t) # Correct tokenisation
    t = re.sub("- |— |°", "", t) # Remove line breaks
    t = re.sub("ABOGADOS|INGENIEROS|AVIADORES|ECLESIASTICOS|ECONOMISTAS|EDUCADORES|POETAS|ESCRITORES, |PERIODISTAS, ", "", t)
    t = re.sub("|".join([pg+ " " + book_title[:i] for i in np.arange(5,15)]+ [str(int(pg)+1)+ " " +  book_title[:i] for i in np.arange(5,15)]+ perm), "", t) # Remove header
    t = re.sub("Na dió|Na  dió", "Nació", t) # Correct OCR mistake
    caps = re.findall("([\p{Lu}]{2,}(?=\s[\p{Lu}\.\,-])(?:\s[\p{Lu}\.\,-]+)+)", t)
    for n in caps:
        hyphen = n.split("-")
        if len(hyphen)==2:
            if len(hyphen[1])<=1:
                n = hyphen[0]
        if fuzz.partial_ratio(n, book_title) >80:
            print(f"Same as header: {n}")
            t.replace(n, " ")
        elif fuzz.partial_ratio(n, book_title)>= 50:
            if len(n)>10:
                print(f"Almost same as header:{n}")
                n = np.array(n.split(" "))
                sim = np.array([fuzz.partial_ratio(word, book_title) for word in n])
                t.replace(" |".join(n[np.where(sim>80)]), " ") # Remove words from header
                n = n[np.where(sim<=80)]
                full_name.append(" ".join(n))
                page_no.append(pg)
                print(f"Corrected: {' '.join(n)}")
            else:
                t.replace(n, " ")
                print(f"Rejected: {n}")
        else:
            n = n.split(" ")
            if len(n) >1:
                if len(n[-1])<=1:
                    n.pop()
                else:
                    n[-1] = n[-1].split(".")[0]
                if len(n) >1:
                    n = " ".join(n)
                    full_name.append(n.strip(". "))
                    page_no.append(pg)
            else:
                t.replace(n, " ")            

    full_text += t + " "

Rejected: IX C
Rejected: XI C
Rejected: IX C
Almost same as header:BRUNAL ECHENIQUE ANTONIO MIGUEL. A
Corrected: BRUNAL ECHENIQUE ANTONIO MIGUEL.
Rejected: II C
Rejected: IV C
Rejected: IX C
Rejected: II A
Rejected: VI A
Rejected: II S
Almost same as header:HENAO HENAO
Corrected: HENAO HENAO
Rejected: LX C
Rejected: II C
Rejected: II C
Rejected: II C
Rejected: XI C
Rejected: II R
Almost same as header:JORGE QUIJANO, C
Corrected: JORGE QUIJANO,
Almost same as header:QUINTERO DE FEX TEOFILO.. A
Corrected: QUINTERO DE FEX TEOFILO..
Rejected: IX C
Rejected: IX C
Almost same as header:URIBE REYENRIQUE
Corrected: URIBE REYENRIQUE
Rejected: II C
Rejected: IX C
Rejected: IX C
Rejected: IX C
Rejected: IV C
Rejected: II A
Rejected: XI S
Rejected: II P
Rejected: IX C
Almost same as header:PIMIENTO JOSE LO
Corrected: PIMIENTO JOSE
Almost same as header:URIBE URIBE LORENZO..
Corrected: URIBE URIBE LORENZO..
Rejected: II C
Rejected: IV C
Rejected: II C
Rejected: IV C
Rejected: IV C
Rejected: IX C
Re

In [37]:
spanish = SpellChecker(language = "es") # initialise language 

# Load city and admin names 
cities = pd.read_csv("cities_colombia.csv") 
spanish.word_frequency.load_words(cities["city"])
spanish.word_frequency.load_words(np.unique(cities["admin_name"]))

# Load category names
cat = ["Nació", "Esposa", "Hijos", "Ha sido","Miembro", "Bachillerato", "Estudios profesionales", 
        "Estudios de literatura", "Estudios secundarios", "Tesis", "Residencia", "Oficina", 
        "Cables", "Telegrafo", "Telefono", "Visitados", "Idiomas", "Afición", "Deportes", "Gabinete", 
        "Condecoraciones", "Especialidad", "Ha visitado", "Paises visitados"]
spanish.word_frequency.load_words(cat)

tkn = nltk.RegexpTokenizer(r'[^\W\d_]+') # Only unicode characters

In [38]:
def autocorrect(text):
    """Correct OCR errors

    Input:
    text(str): Text to be corrected

    Returns:
    text(str): Corrected text
    error(float): Percentage error in the text

    """
    # Split year range
    numbers = re.findall("[\d]{6}[^\d\W]]", text)
    for n in numbers:
        if len(n) == 6:
            text.replace(n, n[:4] + "-"+ n[-2:])
        elif len(n)==8:
            text.replace(n, n[:4]+"-"+n[-4:])

    # Get all unique words
    words = tkn.tokenize(text)
    freq_dict = nltk.FreqDist(words)
    unique_words = list(freq_dict.keys())

    # Find misspelled words and their corrections
    misspelled = spanish.unknown(unique_words)
    correction = [spanish.correction(word) for word in misspelled]
    d = dict(zip(misspelled, correction))

    # Determine how accurate OCR is
    num_mistakes = [freq_dict[word] for word in misspelled]
    error = sum(num_mistakes)/len(text)*100

    # Correct text
    for wrong, correct in d.items():
        text = text.replace(wrong.lower(), correct)

    return text, error

In [39]:
def section_text(name_list, text):
    """"
    Split text into sections by full name and removes 
    names which don't represent entries in the book.

    Input: 
    name_list(list): List of candidate names
    text(str): Full OCR text
    
    Returns: 
    section(list): Sectioned text
    """
    section = []
    txt = text
    for name in name_list:
        split_text = txt.split(name,1)
        section.append(split_text[0])
        txt = split_text[1]

    section.append(txt)
    section.pop(0)

    return section

In [40]:
rmv = full_name.index("LA DEFENSA")
full_name.pop(rmv)
page_no.pop(rmv)

# Remove capitalised letters within section text
keep_index = [i for i, t in enumerate(section_text(full_name, full_text)) if len(t)>50]
full_name = np.array(full_name)[keep_index]
page_no = np.array(page_no)[keep_index]

# Improve OCR Output

In [41]:
# Correct text
sectioned_text = []
error_rate = []
for s in section_text(full_name, full_text):
    s, mistakes = autocorrect(s)
    sectioned_text.append(s)
    error_rate.append(mistakes)

In [42]:
df = pd.DataFrame(
        {"name": full_name, 
        "page": page_no,
        "section_text":sectioned_text,
        "percent_error": error_rate
        })

In [43]:
df

,name,page,section_text,percent_error
0,ABADIA ARANGO SERGIO,007,.. Abogado. Especialidad: Minas y Petróleos. ...,0.000000
1,ABAUNZA JIMENEZ JOSE C,007,.. Abogado. Especialidad: Derecho Civil.. Naci...,0.000000
2,ACERO PIMENTEL ENRIQUE,007,.. Abogado. Especialidad: Derecho Administrati...,0.086505
3,AGUILERA CAMACHO ALBERTO,007,.. Abogado. Especialidad: Derecho del Trabajo ...,0.000000
4,ALBORNOZ ROSAS CARLOS ALBERTO,008,.. Abogado. Especialidad: Derecho Penal.. Nac...,0.000000
...,...,...,...,...
1182,REDER GRANIER CARLOS,363,".. Odontólogo.. Nació, Budapest, Hungría, feb...",0.136799
1183,RODRIGUEZ BOHORQUEZ RAFAEL MARIA,363,".. Odontólogo.. Nació. Guateque, Boyacá, ma...",0.000000
1184,ROZENTAL LERNER SALVADOR,363,".. Odontólogo. Médico.. Nació, Edinita, Besa...",0.172315
1185,SEPULVEDA C. ANTONIO M,364,.. Odontólogo. Especialidad: Cirugía Oral.. N...,0.000000


In [44]:
df.to_csv("sectioned_text.csv", encoding = 'utf-8-sig', index = False)